In [1]:
# все библиотеки использованные в проекте
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [83]:
# обучающие данные
df_x = pd.read_csv('data/X_train.csv')
df_y = pd.read_csv('data/y_train.csv')

In [86]:
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199861 entries, 0 to 1199860
Data columns (total 49 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   index                       1199861 non-null  int64  
 1   acc_now_delinq              1199861 non-null  float64
 2   addr_state                  1199861 non-null  object 
 3   annual_inc                  1199861 non-null  float64
 4   chargeoff_within_12_mths    1199824 non-null  float64
 5   collections_12_mths_ex_med  1199824 non-null  float64
 6   delinq_2yrs                 1199861 non-null  float64
 7   dti                         1199861 non-null  float64
 8   earliest_cr_line            1199861 non-null  object 
 9   emp_length                  1128114 non-null  float64
 10  fico_range_high             1199861 non-null  float64
 11  fico_range_low              1199861 non-null  float64
 12  funded_amnt                 1199861 non-null  float64
 1

In [8]:
df_x.head()

,index,acc_now_delinq,addr_state,annual_inc,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,dti,earliest_cr_line,emp_length,...,pub_rec,pub_rec_bankruptcies,purpose,revol_util,tax_liens,term,title,total_acc,verification_status,zip_code
0,0,0.0,UT,80000.0,0.0,0.0,0.0,30.49,Oct-1996,4 years,...,1.0,1.0,credit_card,56.1%,0.0,36 months,Credit card refinancing,26.0,Source Verified,847xx
1,1,0.0,CA,82000.0,0.0,0.0,0.0,7.00,Mar-1993,10+ years,...,1.0,1.0,credit_card,88.3%,0.0,36 months,Credit card refinancing,11.0,Not Verified,900xx
2,2,0.0,NV,46080.0,0.0,0.0,1.0,17.32,Nov-2000,3 years,...,0.0,0.0,credit_card,18.1%,0.0,36 months,Credit card refinancing,31.0,Source Verified,895xx
3,3,0.0,AZ,30000.0,0.0,0.0,0.0,7.80,Jan-2010,< 1 year,...,0.0,0.0,car,33.4%,0.0,36 months,Car financing,24.0,Source Verified,853xx
4,4,0.0,OH,70000.0,0.0,0.0,0.0,13.36,Jan-2002,< 1 year,...,0.0,0.0,other,88.6%,0.0,60 months,Other,16.0,Not Verified,453xx


## Приведение даненых к нормальному виду

### 1 - избавление от лишних столбйцов
zip_code - возможно имеет зависимость географическое положение (пока не дропаем)(кодируем)

### Функции преобразования данных

In [68]:
# функция создания глобальных файлов
def create_global_file(df, name_column, name_txt):
    dict_df = {}
    j = 1
    for i in df[name_column].unique().tolist():
        if pd.isna(i):
            pass
        else:
            dict_df[i] = j
            j += 1
    with open(f'global_files/{name_txt}.txt', 'w', encoding='utf-8') as f:
        for x, y in dict_df.items():
            f.write(f'{x} {y}\n')


def load_global_file(file_name, type=1):
    with open(f'{file_name}', 'r', encoding='utf-8') as f:
        res = {}
        for i in f.readlines():
            if type == 1:
                key, value = i.split()
            else:
                s_split = i.split()
                key, value = (''.join(s_split[:-1:])), s_split[-1]

            res[key] = int(value)

    return res


In [77]:
# функции преобразования данных
def convert_verification_status_to_int(x):
    if x == 'Source Verified':
        return 1
    elif x == 'Not Verified':
        return 2
    else:
        return 3
def convert_workyear(x):
    s = ''.join(x.split())
    if s[2] == '+':
        return 10
    elif s[0] == '<':
        return 0
    else:
        return int(s[0])


In [ ]:
# создание глобальных файлов, чтобы новый поток данных кадировлся идентично старым
create_global_file(df_x, 'zip_code', 'zip_code')
create_global_file(df_x, 'addr_state', 'addr_state')
create_global_file(df_x, 'home_ownership', 'home_ownership')
create_global_file(df_x, 'title', 'title')
create_global_file(df_x, 'purpose', 'purpose')
create_global_file(df_x, 'earliest_cr_line_month', 'month')

In [32]:
# пути к глобальным файлам в которых кодируются строки
ZIP_CODE_FILE = 'global_files/zip_code.txt'
ADDR_STATE = 'global_files/addr_state.txt'
HOME_OWNERSHIP = 'global_files/home_ownership.txt'
TITLE = 'global_files/title.txt'
PURPOSE_FILE = 'global_files/purpose.txt'
MONTH_FILE = 'global_files/month.txt'

In [85]:
df_x['term'] = df_x['term'].apply(lambda x: x if pd.isna(x) else int(x.split()[0]))
df_x['emp_length'] = df_x['emp_length'].apply(lambda x: x if pd.isna(x) else convert_workyear(x))
df_x['int_rate'] = df_x['int_rate'].apply(lambda x: x if pd.isna(x) else float(x.split('%')[0]))
df_x['revol_util'] = df_x['revol_util'].apply(lambda x: x if pd.isna(x) else float(x.split('%')[0]))
df_x['issue_d_month'] = df_x.issue_d.apply(lambda x: x if pd.isna(x) else x.split('-')[0])
df_x['issue_d_year'] = df_x.issue_d.apply(lambda x: x if pd.isna(x) else int(x.split('-')[1]))
df_x['earliest_cr_line_month'] = df_x.issue_d.apply(lambda x: x if pd.isna(x) else x.split('-')[0])
df_x['earliest_cr_line_year'] = df_x.issue_d.apply(lambda x: x if pd.isna(x) else int(x.split('-')[1]))
df_x['verification_status'] = df_x['verification_status'].apply(
    lambda x: x if pd.isna(x) else convert_verification_status_to_int(x))


In [80]:
zip_code_dict = load_global_file(ZIP_CODE_FILE)
df_x['zip_code'] = df_x['zip_code'].apply(lambda x: x if pd.isna(x) else zip_code_dict[x])
addr_state_dict = load_global_file(ADDR_STATE)
df_x['addr_state'] = df_x['addr_state'].apply(lambda x: x if pd.isna(x) else addr_state_dict[x])
home_ownership_dict = load_global_file(HOME_OWNERSHIP)
df_x['home_ownership'] = df_x['home_ownership'].apply(lambda x: x if pd.isna(x) else home_ownership_dict[x])
title_dict = load_global_file(TITLE, 'title')
df_x['title'] = df_x['title'].apply(lambda x: x if pd.isna(x) else title_dict[''.join(x.split())])
purpose_dict = load_global_file(PURPOSE_FILE)
df_x['purpose'] = df_x['purpose'].apply(lambda x: x if pd.isna(x) else purpose_dict[x])
month_dict = load_global_file(MONTH_FILE)
df_x['earliest_cr_line_month'] = df_x['earliest_cr_line_month'].apply(lambda x: x if pd.isna(x) else month_dict[x])
df_x['issue_d_month'] = df_x['issue_d_month'].apply(lambda x: x if pd.isna(x) else month_dict[x])

In [81]:
# удаление луже не нужных столбцов
df_x.drop(['issue_d','earliest_cr_line'],axis=1,inplace=True)

In [76]:
df_x['emp_length'].value_counts()

emp_length
10+ years    396911
2 years      109336
3 years       96813
< 1 year      94486
1 year        79958
5 years       75440
4 years       72998
6 years       55398
8 years       51577
7 years       51168
9 years       44029
Name: count, dtype: int64